In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
!ls "/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/"

 clr_callback.py		 'Pet Adoption (CB) - PC.ipynb'
 Dataset			 'Pet Adoption (EDA).ipynb'
 learningratefinder.py		 'Pet Adoption (LGB) - BC.ipynb'
'NPZ Files'			 'Pet Adoption (LGB) - PC.ipynb'
'Pet Adoption (ANN) - BC.ipynb'   Predictions
'Pet Adoption (ANN) - PC.ipynb'   __pycache__
'Pet Adoption (CB) - BC.ipynb'


In [3]:
! pip install optuna

     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 8.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 112kB 16.5MB/s 
     |████████████████████████████████| 133kB 17.0MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=686263ff37b34fbdcc7205429bd7b15a982232a00fb6e3543070a023e84a7a8f
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-2.0.0-cp36-none-any.whl size=312964 sha256=2663346358aa702dee77239e0a8c340170c492940c5cb84586fab418cc4f81b3
  Stored in directory: /root/.cache/pip/wheels/b5/

## Import Packages

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.decomposition import PCA
import featuretools as ft
import lightgbm as lgb
import optuna

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


## Set file paths for train and predict datasets

In [5]:
train_dataset = "/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/Dataset/train.csv"
predict_dataset = "/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/Dataset/test.csv"

## Data Preprocessing

### Read train and predict datasets

In [6]:
train_df = pd.read_csv(train_dataset)
predict_df = pd.read_csv(predict_dataset)
print("train_df: {}".format(train_df.shape))
print("predict_df: {}".format(predict_df.shape))

train_df: (18834, 11)
predict_df: (8072, 9)


### Extract target variables into NumPy arrays

In [7]:
train_y_bc = np.array([train_df['breed_category'].values]).T
train_df.drop(['breed_category','pet_category'], inplace=True, axis=1)
print("train_y_bc: {}".format(train_y_bc.shape))

train_y_bc: (18834, 1)


In [8]:
temp_df = pd.DataFrame(train_y_bc, columns=['Class'])
temp_df.groupby(['Class']).size().reset_index().rename(columns={0:'count'})

,Class,count
0,0.0,9000
1,1.0,8357
2,2.0,1477


### Combine train and predict dataframes

In [9]:
combined_df = train_df.append(predict_df, sort=False, ignore_index=True)
combined_df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4


### Feature Engineering

#### Issue Date and Listing Date

In [10]:
combined_df['iss_dt_year'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).year)
combined_df['iss_dt_quarter'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).quarter)
combined_df['iss_dt_month'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).month)
combined_df['iss_dt_week'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).week)
combined_df['iss_dt_day_year'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).dayofyear)
combined_df['iss_dt_day_month'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).day)
combined_df['iss_dt_day_week'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).dayofweek)
combined_df['iss_dt_day_weekend'] = np.where(combined_df['iss_dt_day_week'].isin([5,6]),1,0)

In [11]:
combined_df['lst_dt_year'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).year)
combined_df['lst_dt_quarter'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).quarter)
combined_df['lst_dt_month'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).month)
combined_df['lst_dt_week'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).week)
combined_df['lst_dt_day_year'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).dayofyear)
combined_df['lst_dt_day_month'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).day)
combined_df['lst_dt_day_week'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).dayofweek)
combined_df['lst_dt_day_weekend'] = np.where(combined_df['lst_dt_day_week'].isin([5,6]),1,0)
combined_df['lst_dt_hour'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).hour)
combined_df['lst_dt_minute'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).minute)
combined_df['lst_dt_minutes_elapsed'] = combined_df['listing_date'].map(lambda x: (pd.to_datetime(x) - pd.to_datetime(pd.to_datetime(x).date())).seconds / 60.0)

In [12]:
combined_df['dt_diff'] = (pd.to_datetime(combined_df['listing_date']) - pd.to_datetime(combined_df['issue_date']))
combined_df['dt_diff_days'] = combined_df['dt_diff']/np.timedelta64(1,'D')
combined_df['dt_diff_weeks'] = combined_df['dt_diff']/np.timedelta64(1,'W')
combined_df['dt_diff_mnths'] = combined_df['dt_diff']/np.timedelta64(1,'M')
combined_df['dt_diff_yrs'] = combined_df['dt_diff']/np.timedelta64(1,'Y')
combined_df['dt_diff_sec'] = combined_df['dt_diff']/np.timedelta64(1,'s')
combined_df['dt_diff_min'] = combined_df['dt_diff']/np.timedelta64(1,'m')
combined_df['dt_diff_hrs'] = combined_df['dt_diff']/np.timedelta64(1,'h')

In [13]:
combined_df['curr_iss_dt_diff'] = (pd.datetime.now() - pd.to_datetime(combined_df['issue_date']))
combined_df['curr_iss_dt_diff_mnths'] = combined_df['curr_iss_dt_diff']/np.timedelta64(1,'M')
combined_df['curr_iss_dt_diff_yrs'] = combined_df['curr_iss_dt_diff']/np.timedelta64(1,'Y')

combined_df['curr_lst_dt_diff'] = (pd.datetime.now() - pd.to_datetime(combined_df['listing_date']))
combined_df['curr_lst_dt_diff_mnths'] = combined_df['curr_lst_dt_diff']/np.timedelta64(1,'M')
combined_df['curr_lst_dt_diff_yrs'] = combined_df['curr_lst_dt_diff']/np.timedelta64(1,'Y')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.



In [14]:
combined_df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73 days 16:25:00,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,1504 days 07:52:51.290542,49.424534,4.118711,1430 days 15:27:51.311528,47.003654,3.916971
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862 days 17:47:00,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,2466 days 07:52:51.290542,81.030933,6.752578,603 days 14:05:51.311528,19.830794,1.652566
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752 days 08:24:00,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,2155 days 07:52:51.290542,70.813064,5.901089,1402 days 23:28:51.311528,46.094692,3.841224
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755 days 18:30:00,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,1330 days 07:52:51.290542,43.707784,3.642315,574 days 13:22:51.311528,18.877021,1.573085
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52 days 09:38:00,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,1059 days 07:52:51.290542,34.804111,2.900343,1006 days 22:14:51.311528,33.082469,2.756872


#### Condition

In [15]:
combined_df['condition'] = combined_df['condition'].fillna(value=3)

In [16]:
dummy_val = pd.get_dummies(combined_df['condition'], prefix='condition')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

In [17]:
combined_df['condition_odd_even'] = combined_df['condition'].apply(lambda x: 'EVEN' if x in [0, 2] else 'ODD' if x == 1 else 'NONE')
dummy_val = pd.get_dummies(combined_df['condition_odd_even'], prefix='condition_odd_even')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

In [18]:
combined_df['condition*X1'] = np.cbrt(combined_df['condition'] * combined_df['X1'])
combined_df['condition*X2'] = combined_df['condition'] * combined_df['X2']

In [19]:
combined_df['condition%X1'] = combined_df.apply(lambda row: 0 if row['condition']==0 else row['X1']%row['condition'], axis=1)
combined_df['condition%X2'] = combined_df.apply(lambda row: 0 if row['condition']==0 else row['X2']%row['condition'], axis=1)

#### Pet Id

In [20]:
combined_df['pet_id_num'] = combined_df['pet_id'].apply(lambda x: x.split('_')[1]).astype(int)
combined_df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_0.0,condition_1.0,condition_2.0,condition_3.0,condition_odd_even,condition_odd_even_EVEN,condition_odd_even_NONE,condition_odd_even_ODD,condition*X1,condition*X2,condition%X1,condition%X2,pet_id_num
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73 days 16:25:00,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,1504 days 07:52:51.290542,49.424534,4.118711,1430 days 15:27:51.311528,47.003654,3.916971,0,0,1,0,EVEN,1,0,0,2.962496,18.0,1.0,1.0,69903
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862 days 17:47:00,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,2466 days 07:52:51.290542,81.030933,6.752578,603 days 14:05:51.311528,19.830794,1.652566,0,1,0,0,ODD,0,0,1,2.351335,9.0,0.0,0.0,66892
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,3.0,Brown,0.15,40.90,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752 days 08:24:00,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,2155 days 07:52:51.290542,70.813064,5.901089,1402 days 23:28:51.311528,46.094692,3.841224,0,0,0,1,NONE,0,1,0,3.556893,12.0,0.0,1.0,69750
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755 days 18:30:00,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,1330 days 07:52:51.290542,43.707784,3.642315,574 days 13:22:51.311528,18.877021,1.573085,0,1,0,0,ODD,0,0,1,0.000000,1.0,0.0,0.0,71623
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52 days 09:38:00,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,1059 days 07:52:51.290542,34.804111,2.900343,1006 days 22:14:51.311528,33.082469,2.756872,0,0,1,0,EVEN,1,0,0,3.301927,8.0,0.0,0.0,57969


In [21]:
combined_df['pet_id_odd'] = combined_df['pet_id_num'].apply(lambda x: 0 if x%2==0 else 1)
combined_df['pet_id_even'] = combined_df['pet_id_num'].apply(lambda x: 1 if x%2==0 else 0)
combined_df['petid%X1'] = combined_df.apply(lambda row: 0 if row['X1']==0 else row['pet_id_num']%row['X1'], axis=1)
combined_df['petid%X2'] = combined_df.apply(lambda row: 0 if row['X2']==0 else row['pet_id_num']%row['X2'], axis=1)
combined_df['petid%condition'] = combined_df.apply(lambda row: 0 if row['condition']==0 else row['pet_id_num']%row['condition'], axis=1)

In [22]:
combined_df['pet_id_0'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[0]))
combined_df['pet_id_1'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[1]))
combined_df['pet_id_2'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[2]))
combined_df['pet_id_3'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[3]))
combined_df['pet_id_4'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[4]))

In [23]:
combined_df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_0.0,condition_1.0,condition_2.0,condition_3.0,condition_odd_even,condition_odd_even_EVEN,condition_odd_even_NONE,condition_odd_even_ODD,condition*X1,condition*X2,condition%X1,condition%X2,pet_id_num,pet_id_odd,pet_id_even,petid%X1,petid%X2,petid%condition,pet_id_0,pet_id_1,pet_id_2,pet_id_3,pet_id_4
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73 days 16:25:00,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,1504 days 07:52:51.290542,49.424534,4.118711,1430 days 15:27:51.311528,47.003654,3.916971,0,0,1,0,EVEN,1,0,0,2.962496,18.0,1.0,1.0,69903,1,0,2,0,1.0,6,9,9,0,3
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862 days 17:47:00,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,2466 days 07:52:51.290542,81.030933,6.752578,603 days 14:05:51.311528,19.830794,1.652566,0,1,0,0,ODD,0,0,1,2.351335,9.0,0.0,0.0,66892,0,1,7,4,0.0,6,6,8,9,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,3.0,Brown,0.15,40.90,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752 days 08:24:00,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,2155 days 07:52:51.290542,70.813064,5.901089,1402 days 23:28:51.311528,46.094692,3.841224,0,0,0,1,NONE,0,1,0,3.556893,12.0,0.0,1.0,69750,0,1,0,2,0.0,6,9,7,5,0
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755 days 18:30:00,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,1330 days 07:52:51.290542,43.707784,3.642315,574 days 13:22:51.311528,18.877021,1.573085,0,1,0,0,ODD,0,0,1,0.000000,1.0,0.0,0.0,71623,1,0,0,0,0.0,7,1,6,2,3
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52 days 09:38:00,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,1059 days 07:52:51.290542,34.804111,2.900343,1006 days 22:14:51.311528,33.082469,2.756872,0,0,1,0,EVEN,1,0,0,3.301927,8.0,0.0,0.0,57969,1,0,9,1,1.0,5,7,9,6,9


#### Length and Height

In [24]:
combined_df['height(m)'] = combined_df['height(cm)']/100

In [25]:
combined_df['length_scale_10'] = np.array(np.round((combined_df['length(m)'] * 10)), dtype='int')
combined_df['length_scale_100'] = np.array(np.round((combined_df['length(m)'] * 100)), dtype='int')

combined_df['height_scale_10'] = np.array(np.round((combined_df['height(m)'] * 10)), dtype='int')
combined_df['height_scale_100'] = np.array(np.round((combined_df['height(m)'] * 100)), dtype='int')
combined_df['height_scale_1000'] = np.array(np.round((combined_df['height(m)'] * 1000)), dtype='int')

In [26]:
combined_df['diag(cm)'] = np.sqrt(combined_df['length(m)']**2 + combined_df['height(m)']**2)*100
combined_df['cbrt(m)'] = np.cbrt(combined_df['length(m)']**2 + combined_df['height(m)']**2)
combined_df['area(m)'] = np.sqrt(combined_df['length(m)'] * combined_df['height(m)'])

In [27]:
combined_df['l=h'] = combined_df.apply(lambda row: 1 if np.round(row['length(m)'],1)==np.round(row['height(m)'],1) else 0, axis=1)
combined_df['l>h'] = combined_df.apply(lambda row: 1 if np.round(row['length(m)'],1)>np.round(row['height(m)'],1) else 0, axis=1)
combined_df['l<h'] = combined_df.apply(lambda row: 1 if np.round(row['length(m)'],1)<np.round(row['height(m)'],1) else 0, axis=1)

In [28]:
combined_df['pet_size'] = combined_df['diag(cm)'].apply(lambda x: 'SMALL' if x<=40 else 'MEDIUM' if x>40 and x<=80 else 'BIG')
dummy_val = pd.get_dummies(combined_df['pet_size'], prefix='pet_size')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

#### Color Type

In [29]:
combined_df['subcolor_type1'] = combined_df['color_type'].apply(lambda x: x.split()[0])
dummy_val = pd.get_dummies(combined_df['subcolor_type1'], prefix='subcolor_type1')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

In [30]:
combined_df['subcolor_type2'] = combined_df['color_type'].apply(lambda x: 'NONE' if len(x.split())==1 else x.split()[1])
dummy_val = pd.get_dummies(combined_df['subcolor_type2'], prefix='subcolor_type2')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

In [31]:
combined_df["color_type_num_words"] = combined_df["color_type"].apply(lambda x: len(str(x).split()))
combined_df["color_type_num_chars"] = combined_df["color_type"].apply(lambda x: len(str(x)))

In [32]:
tfidf = TfidfVectorizer(analyzer='char')
features = tfidf.fit_transform(combined_df.color_type).toarray()
features_df = pd.DataFrame(features, columns=tfidf.get_feature_names())
print(features_df.shape)
combined_df = pd.merge(combined_df, features_df, left_index=True, right_index=True)

(26906, 24)


In [33]:
countvec = CountVectorizer(analyzer='char')
features = countvec.fit_transform(combined_df.color_type).toarray()
features_df = pd.DataFrame(features, columns=countvec.get_feature_names())
print(features_df.shape)
combined_df = pd.merge(combined_df, features_df, left_index=True, right_index=True)

(26906, 24)


#### X1 and X2

In [34]:
dummy_val = pd.get_dummies(combined_df['X1'], prefix='X1')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

dummy_val = pd.get_dummies(combined_df['X2'], prefix='X2')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

In [35]:
combined_df['X1_odd_even'] = combined_df['X1'].apply(lambda x: 'EVEN' if x%2==0 else 'ODD')
dummy_val = pd.get_dummies(combined_df['X1_odd_even'], prefix='X1_odd_even')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

combined_df['X2_odd_even'] = combined_df['X2'].apply(lambda x: 'EVEN' if x%2==0 else 'ODD')
dummy_val = pd.get_dummies(combined_df['X2_odd_even'], prefix='X2_odd_even')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

In [36]:
combined_df['Xavg'] = (combined_df['X1']+combined_df['X2'])/2
combined_df['Xdiff'] = np.abs(combined_df['X1']-combined_df['X2'])
combined_df['Xmul'] = combined_df['X1']*combined_df['X2']
combined_df['Xdiv'] = combined_df.apply(lambda row: 0 if row['X1']==0 or row['X2']==0 else row['X2']/row['X1'], axis=1)
combined_df['Xdiv'] = np.cbrt(combined_df['Xdiv'])
combined_df['Xmod'] = combined_df.apply(lambda row: 0 if row['X1']==0 or row['X2']==0 else row['X1']%row['X2'], axis=1)
combined_df['Xdiag'] = np.sqrt(combined_df['X1']**2 + combined_df['X2']**2)
combined_df['Xcbrt'] = np.cbrt(combined_df['X1']**2 + combined_df['X2']**2)

In [37]:
combined_df['X1_cat'] = combined_df['X1'].apply(lambda x: '<=6' if x<=6 else '6to13' if x>6 and x<=13 else '>13')
dummy_val = pd.get_dummies(combined_df['X1_cat'], prefix='X1_cat')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

combined_df['X2_cat'] = combined_df['X2'].apply(lambda x: '<=3' if x<=3 else '4to7' if x>3 and x<=7 else '>7')
dummy_val = pd.get_dummies(combined_df['X2_cat'], prefix='X2_cat')
combined_df = pd.concat([combined_df, dummy_val], axis=1)

In [38]:
combined_df['X1=X2'] = combined_df.apply(lambda row: 1 if row['X1']==row['X2'] else 0, axis=1)
combined_df['X1>X2'] = combined_df.apply(lambda row: 1 if row['X1']>row['X2'] else 0, axis=1)
combined_df['X1<X2'] = combined_df.apply(lambda row: 1 if row['X1']<row['X2'] else 0, axis=1)

In [39]:
#combined_df['length/X1'] = combined_df.apply(lambda row: 0 if row['length(m)']==0 or row['X1']==0 else row['length(m)']/row['X1'], axis=1)
combined_df['length/X2'] = combined_df.apply(lambda row: 0 if row['length(m)']==0 or row['X2']==0 else row['length(m)']/row['X2'], axis=1)
combined_df['length/Xavg'] = combined_df.apply(lambda row: 0 if row['length(m)']==0 or row['Xavg']==0 else np.log1p(row['length(m)']/row['Xavg']), axis=1)
combined_df['length/Xdiag'] = combined_df.apply(lambda row: 0 if row['length(m)']==0 or row['Xdiag']==0 else np.log1p(row['length(m)']/row['Xdiag']), axis=1)

In [40]:
#combined_df['height/X1'] = combined_df.apply(lambda row: 0 if row['height(m)']==0 or row['X1']==0 else row['height(m)']/row['X1'], axis=1)
combined_df['height/X2'] = combined_df.apply(lambda row: 0 if row['height(m)']==0 or row['X2']==0 else row['height(m)']/row['X2'], axis=1)
combined_df['height/Xavg'] = combined_df.apply(lambda row: 0 if row['height(m)']==0 or row['Xavg']==0 else np.log1p(row['height(m)']/row['Xavg']), axis=1)
combined_df['height/Xdiag'] = combined_df.apply(lambda row: 0 if row['height(m)']==0 or row['Xdiag']==0 else np.log1p(row['length(m)']/row['Xdiag']), axis=1)

In [41]:
#combined_df['diag/X1'] = combined_df.apply(lambda row: 0 if row['diag(cm)']==0 or row['X1']==0 else row['diag(cm)']/row['X1'], axis=1)
combined_df['diag/X2'] = combined_df.apply(lambda row: 0 if row['diag(cm)']==0 or row['X2']==0 else row['diag(cm)']/row['X2'], axis=1)
combined_df['diag/Xavg'] = combined_df.apply(lambda row: 0 if row['diag(cm)']==0 or row['Xavg']==0 else np.log1p(row['diag(cm)']/row['Xavg']), axis=1)
combined_df['diag/Xdiag'] = combined_df.apply(lambda row: 0 if row['diag(cm)']==0 or row['Xdiag']==0 else np.log1p(row['diag(cm)']/row['Xdiag']), axis=1)

In [42]:
#combined_df['area/X1'] = combined_df.apply(lambda row: 0 if row['area(m)']==0 or row['X1']==0 else row['area(m)']/row['X1'], axis=1)
combined_df['area/X2'] = combined_df.apply(lambda row: 0 if row['area(m)']==0 or row['X2']==0 else row['area(m)']/row['X2'], axis=1)
combined_df['area/Xavg'] = combined_df.apply(lambda row: 0 if row['area(m)']==0 or row['Xavg']==0 else np.log1p(row['area(m)']/row['Xavg']), axis=1)
combined_df['area/Xdiag'] = combined_df.apply(lambda row: 0 if row['area(m)']==0 or row['Xdiag']==0 else np.log1p(row['area(m)']/row['Xdiag']), axis=1)

#### Drop redundant fields

In [43]:
combined_df.drop(['condition_odd_even','color_type', 'height(cm)', 'issue_date', 'listing_date', 'dt_diff', 'X1_cat', 'X2_cat', 'X1_odd_even', 
                  'X2_odd_even', 'pet_size', 'subcolor_type1', 'subcolor_type2', 'curr_iss_dt_diff', 'curr_lst_dt_diff'], axis=1, inplace=True)
combined_df.head()

,pet_id,condition,length(m),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_0.0,condition_1.0,condition_2.0,condition_3.0,condition_odd_even_EVEN,...,X2_2,X2_3,X2_4,X2_5,X2_6,X2_7,X2_8,X2_9,X1_odd_even_EVEN,X1_odd_even_ODD,X2_odd_even_EVEN,X2_odd_even_ODD,Xavg,Xdiff,Xmul,Xdiv,Xmod,Xdiag,Xcbrt,X1_cat_6to13,X1_cat_<=6,X1_cat_>13,X2_cat_4to7,X2_cat_<=3,X2_cat_>7,X1=X2,X1>X2,X1<X2,length/X2,length/Xavg,length/Xdiag,height/X2,height/Xavg,height/Xdiag,diag/X2,diag/Xavg,diag/Xdiag,area/X2,area/Xavg,area/Xdiag
0,ANSL_69903,2.0,0.80,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.424534,4.118711,47.003654,3.916971,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,1,0,1,11.0,4,117,0.884640,4,15.811388,6.299605,1,0,0,0,0,1,0,1,0,0.088889,0.070204,0.049358,0.008644,0.007048,0.049358,8.930824,2.117103,1.805582,0.027720,0.022427,0.015655
1,ANSL_66892,1.0,0.72,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.030933,6.752578,19.830794,1.652566,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,1,11.0,4,117,0.884640,4,15.811388,6.299605,1,0,0,0,0,1,0,1,0,0.080000,0.063402,0.044530,0.015767,0.012818,0.044530,8.153887,2.037494,1.730110,0.035515,0.028644,0.020014
2,ANSL_69750,3.0,0.15,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.813064,5.901089,46.094692,3.841224,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1,1,0,9.5,11,60,0.643660,3,15.524175,6.223084,0,0,1,1,0,0,0,1,0,0.037500,0.015666,0.009616,0.102250,0.042152,0.009616,10.890965,1.720204,1.336630,0.061922,0.025738,0.015829
3,ANSL_71623,1.0,0.62,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.707784,3.642315,18.877021,1.573085,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0.5,1,0,0.000000,0,1.000000,1.000000,0,1,0,0,1,0,0,0,1,0.620000,0.806476,0.482426,0.178200,0.304834,0.482426,64.510095,4.867690,4.182204,0.332391,0.509695,0.286975
4,ANSL_57969,2.0,0.50,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,34.804111,2.900343,33.082469,2.756872,0,0,1,0,1,...,0,0,1,0,0,0,0,0,1,0,1,0,11.0,14,72,0.605707,2,18.439089,6.979532,0,0,1,1,0,0,0,1,0,0.125000,0.044452,0.026755,0.027650,0.010004,0.026755,12.802157,1.732598,1.328977,0.058790,0.021153,0.012673


#### Feature Tools

In [44]:
feature_df = pd.DataFrame()
feature_df = combined_df[['pet_id','height(m)','length(m)','diag(cm)','area(m)','pet_id_num','petid%X1','petid%X2',
                          'length_scale_10','length_scale_100','height_scale_10','height_scale_100','height_scale_1000',
                          'pet_id_0','pet_id_1','pet_id_2','pet_id_3','pet_id_4',
                          'Xavg','Xdiff','Xmul','Xdiv','Xmod','Xdiag','Xcbrt','condition','X1','X2']]
feature_df.head()

,pet_id,height(m),length(m),diag(cm),area(m),pet_id_num,petid%X1,petid%X2,length_scale_10,length_scale_100,height_scale_10,height_scale_100,height_scale_1000,pet_id_0,pet_id_1,pet_id_2,pet_id_3,pet_id_4,Xavg,Xdiff,Xmul,Xdiv,Xmod,Xdiag,Xcbrt,condition,X1,X2
0,ANSL_69903,0.0778,0.80,80.377412,0.249479,69903,2,0,8,80,1,8,78,6,9,9,0,3,11.0,4,117,0.884640,4,15.811388,6.299605,2.0,13,9
1,ANSL_66892,0.1419,0.72,73.384986,0.319637,66892,7,4,7,72,1,14,142,6,6,8,9,2,11.0,4,117,0.884640,4,15.811388,6.299605,1.0,13,9
2,ANSL_69750,0.4090,0.15,43.563861,0.247689,69750,0,2,2,15,4,41,409,6,9,7,5,0,9.5,11,60,0.643660,3,15.524175,6.223084,3.0,15,4
3,ANSL_71623,0.1782,0.62,64.510095,0.332391,71623,0,0,6,62,2,18,178,7,1,6,2,3,0.5,1,0,0.000000,0,1.000000,1.000000,1.0,0,1
4,ANSL_57969,0.1106,0.50,51.208628,0.235160,57969,9,1,5,50,1,11,111,5,7,9,6,9,11.0,14,72,0.605707,2,18.439089,6.979532,2.0,18,4


In [45]:
es = ft.EntitySet(id='Buddy')
es.entity_from_dataframe(entity_id='Breed_Category', dataframe=feature_df, index='pet_id')
es = es.normalize_entity(base_entity_id='Breed_Category', new_entity_id='condition', index='condition')
es = es.normalize_entity(base_entity_id='Breed_Category', new_entity_id='X1', index='X1')
es = es.normalize_entity(base_entity_id='Breed_Category', new_entity_id='X2', index='X2')
feature_matrix, feature_names = ft.dfs(entityset=es, target_entity='Breed_Category', max_depth=2, verbose=3, n_jobs=1)
print(feature_matrix.shape)
feature_matrix.head()

Built 474 features
Elapsed: 00:19 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks
(26906, 474)


,height(m),length(m),diag(cm),area(m),pet_id_num,petid%X1,petid%X2,length_scale_10,length_scale_100,height_scale_10,height_scale_100,height_scale_1000,pet_id_0,pet_id_1,pet_id_2,pet_id_3,pet_id_4,Xavg,Xdiff,Xmul,Xdiv,Xmod,Xdiag,Xcbrt,condition,X1,X2,condition.SUM(Breed_Category.height(m)),condition.SUM(Breed_Category.length(m)),condition.SUM(Breed_Category.diag(cm)),condition.SUM(Breed_Category.area(m)),condition.SUM(Breed_Category.pet_id_num),condition.SUM(Breed_Category.petid%X1),condition.SUM(Breed_Category.petid%X2),condition.SUM(Breed_Category.length_scale_10),condition.SUM(Breed_Category.length_scale_100),condition.SUM(Breed_Category.height_scale_10),condition.SUM(Breed_Category.height_scale_100),condition.SUM(Breed_Category.height_scale_1000),condition.SUM(Breed_Category.pet_id_0),...,X2.MIN(Breed_Category.pet_id_1),X2.MIN(Breed_Category.pet_id_2),X2.MIN(Breed_Category.pet_id_3),X2.MIN(Breed_Category.pet_id_4),X2.MIN(Breed_Category.Xavg),X2.MIN(Breed_Category.Xdiff),X2.MIN(Breed_Category.Xmul),X2.MIN(Breed_Category.Xdiv),X2.MIN(Breed_Category.Xmod),X2.MIN(Breed_Category.Xdiag),X2.MIN(Breed_Category.Xcbrt),X2.MEAN(Breed_Category.height(m)),X2.MEAN(Breed_Category.length(m)),X2.MEAN(Breed_Category.diag(cm)),X2.MEAN(Breed_Category.area(m)),X2.MEAN(Breed_Category.pet_id_num),X2.MEAN(Breed_Category.petid%X1),X2.MEAN(Breed_Category.petid%X2),X2.MEAN(Breed_Category.length_scale_10),X2.MEAN(Breed_Category.length_scale_100),X2.MEAN(Breed_Category.height_scale_10),X2.MEAN(Breed_Category.height_scale_100),X2.MEAN(Breed_Category.height_scale_1000),X2.MEAN(Breed_Category.pet_id_0),X2.MEAN(Breed_Category.pet_id_1),X2.MEAN(Breed_Category.pet_id_2),X2.MEAN(Breed_Category.pet_id_3),X2.MEAN(Breed_Category.pet_id_4),X2.MEAN(Breed_Category.Xavg),X2.MEAN(Breed_Category.Xdiff),X2.MEAN(Breed_Category.Xmul),X2.MEAN(Breed_Category.Xdiv),X2.MEAN(Breed_Category.Xmod),X2.MEAN(Breed_Category.Xdiag),X2.MEAN(Breed_Category.Xcbrt),X2.COUNT(Breed_Category),X2.NUM_UNIQUE(Breed_Category.condition),X2.NUM_UNIQUE(Breed_Category.X1),X2.MODE(Breed_Category.condition),X2.MODE(Breed_Category.X1)
pet_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ANSL_49970,0.4595,0.78,90.528462,0.598674,49970,0,4,8,78,5,46,460,4,9,9,7,0,3.5,7,0,0.00000,0,7.000000,3.659306,2.0,0,7,1666.0789,3047.46,367055.451273,2055.391497,387134965,26087,19155,30508,304746,16822,166580,1666074,35929,...,0,0,0,0,3.5,7,0,0.000000,0,7.000000,3.659306,0.274825,0.500182,60.373899,0.337672,63595.910742,0.000000,3.002539,5.000781,50.018164,2.781641,27.482813,274.821875,5.900000,4.097461,4.489453,4.499023,4.514258,3.500000,7.000000,0.000000,0.000000,0.000000,7.000000,3.659306,5120,4,1,1.0,0
ANSL_49971,0.0938,0.45,45.967210,0.205451,49971,12,3,4,45,1,9,94,4,9,9,7,1,11.0,4,117,0.88464,4,15.811388,6.299605,2.0,13,9,1666.0789,3047.46,367055.451273,2055.391497,387134965,26087,19155,30508,304746,16822,166580,1666074,35929,...,0,0,0,0,6.0,4,27,0.808968,3,9.486833,4.481405,0.273321,0.504561,60.566045,0.339340,63288.935056,6.343724,3.989199,5.049221,50.456112,2.761553,27.325540,273.321438,5.873393,4.052775,4.520372,4.570823,4.479218,11.275636,4.552912,121.961444,0.873979,4.552502,16.281021,6.420832,7314,4,4,2.0,13
ANSL_49972,0.2681,0.17,31.745489,0.213488,49972,0,0,2,17,3,27,268,4,9,9,7,2,0.5,1,0,0.00000,0,1.000000,1.000000,1.0,0,1,2665.1375,4910.57,590283.003307,3299.567293,618074698,14307,12966,49086,491057,26955,266478,2665114,57329,...,0,0,0,0,0.5,1,0,0.000000,0,1.000000,1.000000,0.274817,0.504802,60.717067,0.339898,63459.301341,0.573659,0.000000,5.048071,50.480216,2.774526,27.479807,274.817364,5.881295,4.151161,4.461494,4.453728,4.504088,1.154554,1.947106,1.309107,0.093896,0.000000,2.140722,1.499028,12232,4,3,1.0,0
ANSL_49973,0.3674,0.89,96.285137,0.571827,49973,1,5,9,89,4,37,367,4,9,9,7,3,11.0,4,117,0.88464,4,15.811388,6.299605,2.0,13,9,1666.0789,3047.46,367055.451273,2055.391497,387134965,26087,19155,30508,304746,16822,166580,1666074,35929,...,0,0,0,0,6.0,4,27,0.8089

In [46]:
feature_matrix.drop(['height(m)','length(m)','diag(cm)','area(m)','pet_id_num','petid%X1','petid%X2',
                     'length_scale_10','length_scale_100','height_scale_10','height_scale_100','height_scale_1000',
                     'pet_id_0','pet_id_1','pet_id_2','pet_id_3','pet_id_4',
                     'Xavg','Xdiff','Xmul','Xdiv','Xmod','Xdiag','Xcbrt','condition','X1','X2'], axis=1, inplace=True)
feature_df = feature_matrix.copy()

In [47]:
combined_df.set_index('pet_id', inplace = True)
combined_df.drop(['condition','pet_id_num'], axis=1, inplace=True)
print(combined_df.shape)
combined_df.head()

(26906, 220)


,length(m),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_0.0,condition_1.0,condition_2.0,condition_3.0,condition_odd_even_EVEN,condition_odd_even_NONE,condition_odd_even_ODD,...,X2_2,X2_3,X2_4,X2_5,X2_6,X2_7,X2_8,X2_9,X1_odd_even_EVEN,X1_odd_even_ODD,X2_odd_even_EVEN,X2_odd_even_ODD,Xavg,Xdiff,Xmul,Xdiv,Xmod,Xdiag,Xcbrt,X1_cat_6to13,X1_cat_<=6,X1_cat_>13,X2_cat_4to7,X2_cat_<=3,X2_cat_>7,X1=X2,X1>X2,X1<X2,length/X2,length/Xavg,length/Xdiag,height/X2,height/Xavg,height/Xdiag,diag/X2,diag/Xavg,diag/Xdiag,area/X2,area/Xavg,area/Xdiag
pet_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ANSL_69903,0.80,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.424534,4.118711,47.003654,3.916971,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1,0,1,11.0,4,117,0.884640,4,15.811388,6.299605,1,0,0,0,0,1,0,1,0,0.088889,0.070204,0.049358,0.008644,0.007048,0.049358,8.930824,2.117103,1.805582,0.027720,0.022427,0.015655
ANSL_66892,0.72,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.030933,6.752578,19.830794,1.652566,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1,0,1,11.0,4,117,0.884640,4,15.811388,6.299605,1,0,0,0,0,1,0,1,0,0.080000,0.063402,0.044530,0.015767,0.012818,0.044530,8.153887,2.037494,1.730110,0.035515,0.028644,0.020014
ANSL_69750,0.15,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.813064,5.901089,46.094692,3.841224,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,1,1,0,9.5,11,60,0.643660,3,15.524175,6.223084,0,0,1,1,0,0,0,1,0,0.037500,0.015666,0.009616,0.102250,0.042152,0.009616,10.890965,1.720204,1.336630,0.061922,0.025738,0.015829
ANSL_71623,0.62,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.707784,3.642315,18.877021,1.573085,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,1,0.5,1,0,0.000000,0,1.000000,1.000000,0,1,0,0,1,0,0,0,1,0.620000,0.806476,0.482426,0.178200,0.304834,0.482426,64.510095,4.867690,4.182204,0.332391,0.509695,0.286975
ANSL_57969,0.50,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,34.804111,2.900343,33.082469,2.756872,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0,1,0,11.0,14,72,0.605707,2,18.439089,6.979532,0,0,1,1,0,0,0,1,0,0.125000,0.044452,0.026755,0.027650,0.010004,0.026755,12.802157,1.732598,1.328977,0.058790,0.021153,0.012673


In [48]:
temp_df = pd.merge(combined_df, feature_df, how='inner', on='pet_id', sort=False, suffixes=('_x', '_y'))
print(temp_df.shape)
temp_df.head()

(26906, 667)


,length(m),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_0.0,condition_1.0,condition_2.0,condition_3.0,condition_odd_even_EVEN,condition_odd_even_NONE,condition_odd_even_ODD,...,X2.MIN(Breed_Category.pet_id_1),X2.MIN(Breed_Category.pet_id_2),X2.MIN(Breed_Category.pet_id_3),X2.MIN(Breed_Category.pet_id_4),X2.MIN(Breed_Category.Xavg),X2.MIN(Breed_Category.Xdiff),X2.MIN(Breed_Category.Xmul),X2.MIN(Breed_Category.Xdiv),X2.MIN(Breed_Category.Xmod),X2.MIN(Breed_Category.Xdiag),X2.MIN(Breed_Category.Xcbrt),X2.MEAN(Breed_Category.height(m)),X2.MEAN(Breed_Category.length(m)),X2.MEAN(Breed_Category.diag(cm)),X2.MEAN(Breed_Category.area(m)),X2.MEAN(Breed_Category.pet_id_num),X2.MEAN(Breed_Category.petid%X1),X2.MEAN(Breed_Category.petid%X2),X2.MEAN(Breed_Category.length_scale_10),X2.MEAN(Breed_Category.length_scale_100),X2.MEAN(Breed_Category.height_scale_10),X2.MEAN(Breed_Category.height_scale_100),X2.MEAN(Breed_Category.height_scale_1000),X2.MEAN(Breed_Category.pet_id_0),X2.MEAN(Breed_Category.pet_id_1),X2.MEAN(Breed_Category.pet_id_2),X2.MEAN(Breed_Category.pet_id_3),X2.MEAN(Breed_Category.pet_id_4),X2.MEAN(Breed_Category.Xavg),X2.MEAN(Breed_Category.Xdiff),X2.MEAN(Breed_Category.Xmul),X2.MEAN(Breed_Category.Xdiv),X2.MEAN(Breed_Category.Xmod),X2.MEAN(Breed_Category.Xdiag),X2.MEAN(Breed_Category.Xcbrt),X2.COUNT(Breed_Category),X2.NUM_UNIQUE(Breed_Category.condition),X2.NUM_UNIQUE(Breed_Category.X1),X2.MODE(Breed_Category.condition),X2.MODE(Breed_Category.X1)
pet_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ANSL_69903,0.80,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.424534,4.118711,47.003654,3.916971,0,0,1,0,1,0,0,...,0,0,0,0,6.0,4,27,0.808968,3,9.486833,4.481405,0.273321,0.504561,60.566045,0.339340,63288.935056,6.343724,3.989199,5.049221,50.456112,2.761553,27.325540,273.321438,5.873393,4.052775,4.520372,4.570823,4.479218,11.275636,4.552912,121.961444,0.873979,4.552502,16.281021,6.420832,7314,4,4,2.0,13
ANSL_66892,0.72,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.030933,6.752578,19.830794,1.652566,0,1,0,0,0,0,1,...,0,0,0,0,6.0,4,27,0.808968,3,9.486833,4.481405,0.273321,0.504561,60.566045,0.339340,63288.935056,6.343724,3.989199,5.049221,50.456112,2.761553,27.325540,273.321438,5.873393,4.052775,4.520372,4.570823,4.479218,11.275636,4.552912,121.961444,0.873979,4.552502,16.281021,6.420832,7314,4,4,2.0,13
ANSL_69750,0.15,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.813064,5.901089,46.094692,3.841224,0,0,0,1,0,1,0,...,0,0,0,0,2.0,0,0,0.000000,0,4.000000,2.519842,0.275164,0.511067,61.295001,0.342440,63203.620553,6.961604,1.569735,5.109543,51.106719,2.786561,27.516093,275.166573,5.868436,4.017504,4.515528,4.569170,4.512705,9.436194,11.388481,59.489554,0.665207,2.373235,15.602776,6.180631,1771,4,9,3.0,15
ANSL_71623,0.62,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.707784,3.642315,18.877021,1.573085,0,1,0,0,0,0,1,...,0,0,0,0,0.5,1,0,0.000000,0,1.000000,1.000000,0.274817,0.504802,60.717067,0.339898,63459.301341,0.573659,0.000000,5.048071,50.480216,2.774526,27.479807,274.817364,5.881295,4.151161,4.461494,4.453728,4.504088,1.154554,1.947106,1.309107,0.093896,0.000000,2.140722,1.499028,12232,4,3,1.0,0
ANSL_579

In [49]:
temp_df.fillna(0, inplace=True)
temp_df[temp_df.isin([np.nan, np.inf, -np.inf]).any(1)].head()

,length(m),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_0.0,condition_1.0,condition_2.0,condition_3.0,condition_odd_even_EVEN,condition_odd_even_NONE,condition_odd_even_ODD,...,X2.MIN(Breed_Category.pet_id_1),X2.MIN(Breed_Category.pet_id_2),X2.MIN(Breed_Category.pet_id_3),X2.MIN(Breed_Category.pet_id_4),X2.MIN(Breed_Category.Xavg),X2.MIN(Breed_Category.Xdiff),X2.MIN(Breed_Category.Xmul),X2.MIN(Breed_Category.Xdiv),X2.MIN(Breed_Category.Xmod),X2.MIN(Breed_Category.Xdiag),X2.MIN(Breed_Category.Xcbrt),X2.MEAN(Breed_Category.height(m)),X2.MEAN(Breed_Category.length(m)),X2.MEAN(Breed_Category.diag(cm)),X2.MEAN(Breed_Category.area(m)),X2.MEAN(Breed_Category.pet_id_num),X2.MEAN(Breed_Category.petid%X1),X2.MEAN(Breed_Category.petid%X2),X2.MEAN(Breed_Category.length_scale_10),X2.MEAN(Breed_Category.length_scale_100),X2.MEAN(Breed_Category.height_scale_10),X2.MEAN(Breed_Category.height_scale_100),X2.MEAN(Breed_Category.height_scale_1000),X2.MEAN(Breed_Category.pet_id_0),X2.MEAN(Breed_Category.pet_id_1),X2.MEAN(Breed_Category.pet_id_2),X2.MEAN(Breed_Category.pet_id_3),X2.MEAN(Breed_Category.pet_id_4),X2.MEAN(Breed_Category.Xavg),X2.MEAN(Breed_Category.Xdiff),X2.MEAN(Breed_Category.Xmul),X2.MEAN(Breed_Category.Xdiv),X2.MEAN(Breed_Category.Xmod),X2.MEAN(Breed_Category.Xdiag),X2.MEAN(Breed_Category.Xcbrt),X2.COUNT(Breed_Category),X2.NUM_UNIQUE(Breed_Category.condition),X2.NUM_UNIQUE(Breed_Category.X1),X2.MODE(Breed_Category.condition),X2.MODE(Breed_Category.X1)
pet_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [50]:
combined_df = temp_df.copy()

### Create train and predict Numpy arrays

In [51]:
# Segregate combined_df into train/predict datasets
train_x = combined_df[:18834].values
predict_x = combined_df[18834:].values

print("train_x: {}".format(train_x.shape))
print("predict_x: {}".format(predict_x.shape))

train_x: (18834, 667)
predict_x: (8072, 667)


In [52]:
# Scale the train_x/predict_x arrays
#scaler = MinMaxScaler().fit(train_x)
scaler = StandardScaler().fit(train_x)
train_x_bc = scaler.transform(train_x)
Xpredict = scaler.transform(predict_x)

### Feature Selection

In [53]:
pca = PCA(.999)
pca.fit(train_x_bc)

PCA(copy=True, iterated_power='auto', n_components=0.999, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [54]:
train_x_bc = pca.transform(train_x_bc)
Xpredict = pca.transform(Xpredict)

print("train_x_bc: {} \nXpredict: {}".format(train_x_bc.shape, Xpredict.shape))

train_x_bc: (18834, 100) 
Xpredict: (8072, 100)


### Split training data into train/ validation/ test datasets

In [55]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=10)
for train_index, test_index in sss.split(train_x_bc, train_y_bc):
    Xtrain_bc, Xvalidation_bc = train_x_bc[train_index], train_x_bc[test_index]
    Ytrain_bc, Yvalidation_bc = train_y_bc[train_index], train_y_bc[test_index]

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=20)
for train_index, test_index in sss.split(Xvalidation_bc, Yvalidation_bc):
    Xvalidation_bc, Xtest_bc = Xvalidation_bc[train_index], Xvalidation_bc[test_index]
    Yvalidation_bc, Ytest_bc = Yvalidation_bc[train_index], Yvalidation_bc[test_index]


print("------------------------- Training Dataset -------------------------")
print("Xtrain_bc shape: {}".format(Xtrain_bc.shape))
print("Ytrain_bc shape: {}".format(Ytrain_bc.shape))

print("\n------------------------- Validation Dataset -------------------------")
print("Xvalidation_bc shape: {}".format(Xvalidation_bc.shape))
print("Yvalidation_bc shape: {}".format(Yvalidation_bc.shape))

print("\n------------------------- Test Dataset -------------------------")
print("Xtest_bc shape: {}".format(Xtest_bc.shape))
print("Ytest_bc shape: {}".format(Ytest_bc.shape))

print("\n------------------------- Prediction Dataset -------------------------")
print("Xpredict shape: {}".format(Xpredict.shape))

------------------------- Training Dataset -------------------------
Xtrain_bc shape: (16950, 100)
Ytrain_bc shape: (16950, 1)

------------------------- Validation Dataset -------------------------
Xvalidation_bc shape: (1130, 100)
Yvalidation_bc shape: (1130, 1)

------------------------- Test Dataset -------------------------
Xtest_bc shape: (754, 100)
Ytest_bc shape: (754, 1)

------------------------- Prediction Dataset -------------------------
Xpredict shape: (8072, 100)


### Handling class imbalance

In [56]:
#sm = SMOTE()
sm = ADASYN()
sm_x, sm_y = sm.fit_sample(Xtrain_bc, Ytrain_bc.ravel())
Xtrain_bc = sm_x
Ytrain_bc = np.array([sm_y]).T
print("Class balancing done.")
print("Xtrain_bc shape: {}".format(Xtrain_bc.shape))
print("Ytrain_bc shape: {}".format(Ytrain_bc.shape))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



Class balancing done.
Xtrain_bc shape: (23811, 100)
Ytrain_bc shape: (23811, 1)


In [57]:
temp_df = pd.DataFrame(Ytrain_bc, columns=['Class'])
temp_df.groupby(['Class']).size().reset_index().rename(columns={0:'count'})

,Class,count
0,0.0,8100
1,1.0,7611
2,2.0,8100


In [58]:
temp_df = pd.DataFrame(Yvalidation_bc, columns=['Class'])
temp_df.groupby(['Class']).size().reset_index().rename(columns={0:'count'})

,Class,count
0,0.0,540
1,1.0,501
2,2.0,89


In [59]:
temp_df = pd.DataFrame(Ytest_bc, columns=['Class'])
temp_df.groupby(['Class']).size().reset_index().rename(columns={0:'count'})

,Class,count
0,0.0,360
1,1.0,335
2,2.0,59


## Save the datasets in NPZ file (for reusability)

In [60]:
np.savez_compressed('/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/NPZ Files/Breed_Category_Dataset.npz',
                    Xtrain_bc=Xtrain_bc, Ytrain_bc=Ytrain_bc,
                    Xvalidation_bc=Xvalidation_bc, Yvalidation_bc=Yvalidation_bc,
                    Xtest_bc=Xtest_bc, Ytest_bc=Ytest_bc,
                    Xpredict=Xpredict)

## Load datasets from the NPZ file

In [5]:
processed_dataset = np.load('/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/NPZ Files/Breed_Category_Dataset.npz', allow_pickle=True)
Xtrain_bc, Ytrain_bc = processed_dataset['Xtrain_bc'], processed_dataset['Ytrain_bc']
Xvalidation_bc, Yvalidation_bc = processed_dataset['Xvalidation_bc'], processed_dataset['Yvalidation_bc']
Xtest_bc, Ytest_bc = processed_dataset['Xtest_bc'], processed_dataset['Ytest_bc']
Xpredict = processed_dataset['Xpredict']

print("------------------------- Training Dataset -------------------------")
print("Xtrain_bc shape: {}".format(Xtrain_bc.shape))
print("Ytrain_bc shape: {}".format(Ytrain_bc.shape))

print("\n------------------------- Validation Dataset -------------------------")
print("Xvalidation_bc shape: {}".format(Xvalidation_bc.shape))
print("Yvalidation_bc shape: {}".format(Yvalidation_bc.shape))

print("\n------------------------- Test Dataset -------------------------")
print("Xtest_bc shape: {}".format(Xtest_bc.shape))
print("Ytest_bc shape: {}".format(Ytest_bc.shape))

print("\n------------------------- Prediction Dataset -------------------------")
print("Xpredict shape: {}".format(Xpredict.shape))

------------------------- Training Dataset -------------------------
Xtrain_bc shape: (23811, 100)
Ytrain_bc shape: (23811, 1)

------------------------- Validation Dataset -------------------------
Xvalidation_bc shape: (1130, 100)
Yvalidation_bc shape: (1130, 1)

------------------------- Test Dataset -------------------------
Xtest_bc shape: (754, 100)
Ytest_bc shape: (754, 1)

------------------------- Prediction Dataset -------------------------
Xpredict shape: (8072, 100)


## Breed Category Prediction

### Hyperparameter search using Optuna

In [62]:
def objective(trial):

    params = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 3,
        "is_unbalance": True,
        "max_bin": 512,
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-4, 1.0),
        "num_leaves": trial.suggest_int("num_leaves", 40, 3000),
        "max_depth": trial.suggest_int("max_depth", 6, 12),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.5, 0.7),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.5, 0.7),
        "bagging_freq": trial.suggest_int("bagging_freq", 5, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 15),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 1e-3, 1e-1)
    }

    lgtrain = lgb.Dataset(Xtrain_bc, label=Ytrain_bc.ravel())
    lgvalidation = lgb.Dataset(Xvalidation_bc, label=Yvalidation_bc.ravel())
        
    model = lgb.train(params, lgtrain, valid_sets=[lgvalidation], 
                      num_boost_round=5000, early_stopping_rounds=200, 
                      verbose_eval=False)
    
    y_pred = model.predict(Xtest_bc, num_iteration=model.best_iteration)
    y_pred_binary = np.array([np.argmax(y_pred, axis=1)]).T
    f1 = f1_score(Ytest_bc, y_pred_binary, average='weighted')
    return f1

In [63]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

[I 2020-08-22 07:57:47,420] Trial 0 finished with value: 0.8806573815205508 and parameters: {'learning_rate': 0.0682426952026407, 'lambda_l2': 0.0009176831926412985, 'num_leaves': 1252, 'max_depth': 9, 'feature_fraction': 0.5412369377309842, 'bagging_fraction': 0.5776373985635553, 'bagging_freq': 12, 'min_child_samples': 7, 'min_child_weight': 0.05443238628657281}. Best is trial 0 with value: 0.8806573815205508.
[I 2020-08-22 08:00:32,504] Trial 1 finished with value: 0.8819261406970273 and parameters: {'learning_rate': 0.0030358029130400036, 'lambda_l2': 0.9364894720237764, 'num_leaves': 1658, 'max_depth': 6, 'feature_fraction': 0.6707833945970028, 'bagging_fraction': 0.5534489266158404, 'bagging_freq': 14, 'min_child_samples': 5, 'min_child_weight': 0.026544133533129156}. Best is trial 1 with value: 0.8819261406970273.
[I 2020-08-22 08:05:09,755] Trial 2 finished with value: 0.8806232403192177 and parameters: {'learning_rate': 0.0015937967370261998, 'lambda_l2': 0.0009231368656546731

KeyboardInterrupt: ignored

In [64]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("Value: {}".format(trial.value))

print("Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key, value))

Number of finished trials: 72
Best trial:
Value: 0.896551724137931
Params: 
 learning_rate: 0.017023702906213488
 lambda_l2: 0.0009241543672763387
 num_leaves: 2582
 max_depth: 11
 feature_fraction: 0.6745200173337966
 bagging_fraction: 0.5696624141299601
 bagging_freq: 12
 min_child_samples: 12
 min_child_weight: 0.0039844113682579365


### Build the model for breed-category prediction

In [9]:
# Define model hyperparameters
params = {}
params["objective"] = 'multiclass'
params['metric'] = 'multi_logloss'
params['boosting'] = 'gbdt'
params['num_class'] = 3
params['is_unbalance'] = True
params['max_bin'] = 512
params["learning_rate"] = 0.017
params["lambda_l2"] = 0.000924
params["num_leaves"] = 2582
params["max_depth"] = 11
params["feature_fraction"] = 0.675
params["bagging_fraction"] = 0.57
params["bagging_freq"] = 12
params["bagging_seed"] = 10
params["min_data_in_leaf"] = 12
params["min_child_weight"] = 0.00398
params["verbosity"] = -1
num_rounds = 5000

In [69]:
# Define K-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=30)
y_pred = 0

In [70]:
Xtrain_tmp = np.concatenate((Xtrain_bc, Xvalidation_bc), axis=0)
Ytrain_tmp = np.concatenate((Ytrain_bc, Yvalidation_bc), axis=0)

In [71]:
# Train the model using K-fold
counter = 0

for train, val in kfold.split(Xtrain_tmp, Ytrain_tmp):
    counter += 1

    train_x, train_y = Xtrain_tmp[train], Ytrain_tmp[train]
    val_x, val_y = Xtrain_tmp[val], Ytrain_tmp[val]
    
    lgtrain = lgb.Dataset(train_x, label=train_y.ravel())
    lgvalidation = lgb.Dataset(val_x, label=val_y.ravel())

    model = lgb.train(params, lgtrain, num_rounds, valid_sets=[lgtrain, lgvalidation], 
                      early_stopping_rounds=200, verbose_eval=100)
    pred = model.predict(Xtest_bc, num_iteration=model.best_iteration)

    y_pred += pred

y_pred /= float(counter)

Training until validation scores don't improve for 200 rounds.
[500]	training's multi_logloss: 0.0233338	valid_1's multi_logloss: 0.113114
Early stopping, best iteration is:
[516]	training's multi_logloss: 0.0212587	valid_1's multi_logloss: 0.112556
Training until validation scores don't improve for 200 rounds.
[500]	training's multi_logloss: 0.0235946	valid_1's multi_logloss: 0.116762
Early stopping, best iteration is:
[408]	training's multi_logloss: 0.0359904	valid_1's multi_logloss: 0.115159
Training until validation scores don't improve for 200 rounds.
[500]	training's multi_logloss: 0.0231497	valid_1's multi_logloss: 0.110902
Early stopping, best iteration is:
[523]	training's multi_logloss: 0.0203546	valid_1's multi_logloss: 0.110686
Training until validation scores don't improve for 200 rounds.
[500]	training's multi_logloss: 0.0246826	valid_1's multi_logloss: 0.106869
Early stopping, best iteration is:
[435]	training's multi_logloss: 0.0330906	valid_1's multi_logloss: 0.106167


### Validate the model

In [72]:
# Print log_loss, accuracy and F1-score
y_pred_binary = np.array([np.argmax(y_pred, axis=1)]).T
f1 = f1_score(Ytest_bc, y_pred_binary, average='weighted')
print('Overall F1-Score:', f1)

Overall F1-Score: 0.8859288740828076


In [73]:
print(classification_report(Ytest_bc, y_pred_binary))

              precision    recall  f1-score   support

         0.0       0.88      0.88      0.88       360
         1.0       0.87      0.87      0.87       335
         2.0       1.00      1.00      1.00        59

    accuracy                           0.89       754
   macro avg       0.92      0.92      0.92       754
weighted avg       0.89      0.89      0.89       754



### Train model on entire data

In [10]:
# Define K-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=50)
y_pred_bc = 0

In [11]:
Xtrain_tmp = np.concatenate((Xtrain_bc, Xvalidation_bc, Xtest_bc), axis=0)
Ytrain_tmp = np.concatenate((Ytrain_bc, Yvalidation_bc, Ytest_bc), axis=0)

In [12]:
# Train the model using K-fold
counter = 0

for train, val in kfold.split(Xtrain_tmp, Ytrain_tmp):
    counter += 1

    train_x, train_y = Xtrain_tmp[train], Ytrain_tmp[train]
    val_x, val_y = Xtrain_tmp[val], Ytrain_tmp[val]
    
    lgtrain = lgb.Dataset(train_x, label=train_y.ravel())
    lgvalidation = lgb.Dataset(val_x, label=val_y.ravel())

    model = lgb.train(params, lgtrain, num_rounds, valid_sets=[lgtrain, lgvalidation], 
                      early_stopping_rounds=200, verbose_eval=100)
    pred = model.predict(Xpredict, num_iteration=model.best_iteration)

    y_pred_bc += pred

y_pred_bc /= float(counter)

Training until validation scores don't improve for 200 rounds.
[100]	training's multi_logloss: 0.236534	valid_1's multi_logloss: 0.263712
[200]	training's multi_logloss: 0.0969597	valid_1's multi_logloss: 0.142545
[300]	training's multi_logloss: 0.0587864	valid_1's multi_logloss: 0.11913
[400]	training's multi_logloss: 0.0382006	valid_1's multi_logloss: 0.115038
[500]	training's multi_logloss: 0.0239946	valid_1's multi_logloss: 0.114785
[600]	training's multi_logloss: 0.0136211	valid_1's multi_logloss: 0.116007
Early stopping, best iteration is:
[472]	training's multi_logloss: 0.0273072	valid_1's multi_logloss: 0.114627
Training until validation scores don't improve for 200 rounds.
[100]	training's multi_logloss: 0.23885	valid_1's multi_logloss: 0.265605
[200]	training's multi_logloss: 0.0981893	valid_1's multi_logloss: 0.145322
[300]	training's multi_logloss: 0.0595346	valid_1's multi_logloss: 0.123279
[400]	training's multi_logloss: 0.0391906	valid_1's multi_logloss: 0.120189
[500]	t

## Create submission file

In [13]:
y_pred_bc_binary = np.array([np.argmax(y_pred_bc, axis=1)]).T
predict_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/Dataset/test.csv")
submit_df = pd.DataFrame()
submit_df['pet_id'] = predict_df['pet_id']
submit_df['breed_category'] = y_pred_bc_binary
submit_df.head()

,pet_id,breed_category
0,ANSL_75005,1
1,ANSL_76663,0
2,ANSL_58259,0
3,ANSL_67171,0
4,ANSL_72871,0


In [14]:
submit_df.to_csv("/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/Predictions/predictions_v57_bc_LGB.csv", index=False)